# Model evaluation - Optimal thresholding on default probas
In this notebook, we use a LogReg trained on 1M SIRETs, evaluated on 250k SIRETs.<br>
We compute $f_{\beta}$ scores, balanced accuracy and select thresholds optimally.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# Set logging level to INFO
import logging
logging.getLogger().setLevel(logging.INFO)

# Import required libraries and modules
from datetime import datetime
from pathlib import Path
import pickle
import json
from types import ModuleType

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, fbeta_score
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper

from predictsignauxfaibles.config import IGNORE_NA
from predictsignauxfaibles.data import SFDataset
from predictsignauxfaibles.evaluate import evaluate, make_precision_recall_curve, make_thresholds_from_fbeta, make_thresholds_from_conditions
from predictsignauxfaibles.make_list import merge_models, assign_flag, make_alert
from predictsignauxfaibles.pipelines import run_pipeline
from predictsignauxfaibles.utils import load_conf

## Loading predictions from csv, splitting models

In [ ]:
## LOADING MODELS FROM FILES

model_runs = {
    "default": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/default_20210519-172348",
    "small": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/small_20210519-172421"
}

## DEFAULT MODEL PIPELINE

default = pd.read_csv(f"{model_runs['default']}/predictions.csv")
default_mapper_unpickled =  pickle.load(
    open( f"{model_runs['default']}/model_comp0.pickle",
        "rb"
        )
)
default_mapper = default_mapper_unpickled[1]
default_model_unpickled = pickle.load(
    open( f"{model_runs['default']}/model_comp1.pickle",
        "rb"
        )
)
default_model = default_model_unpickled[1]

default_pp = Pipeline(
    [("transform_dataframe", default_mapper), ("fit_model", default_model)]
)

## SMALL MODEL PIPELINE

small = pd.read_csv(f"{model_runs['small']}/predictions.csv")
small_mapper_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp0.pickle",
        "rb"
        )
)
small_mapper = small_mapper_unpickled[1]
small_model_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp1.pickle",
        "rb"
        )
)
small_model = small_model_unpickled[1]

small_pp = Pipeline(
    [("transform_dataframe", small_mapper), ("fit_model", small_model)]
)

In [ ]:
default.shape

In [ ]:
small.shape

### Making list for the merged model

In [ ]:
merged = merge_models(model_list = [default, small])

## Computing precision/recall graph as a function of classification thresholds - Default model

Inspecting the model from saved pickle

Let's load a test set from which we can use the output to measure our performance

In [ ]:
default_conf = load_conf(model_name="default")
test = default_conf.TEST_DATASET
test.sample_size = 2.5e5

In [ ]:
#test.fetch_data()
test.data = pd.read_csv(f"/home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv")

In [ ]:
test.replace_missing_data().remove_na(ignore=IGNORE_NA)
test.data = run_pipeline(test.data, default_conf.TRANSFO_PIPELINE)

((((((((((((((((((
#### Code for local execution (no calling functions from make_list)

A couple of quicks checks/diagnoses:

In [ ]:
test.data.sample(n=3)

In [ ]:
100*test.data.groupby(by="outcome").siret.count()/len(test)

Isolating transformed features on one hand, outcomes on the other:

In [ ]:
test_features = test.data[set(test.data.columns).difference(set(["outcome"]))]
test_outcomes = test.data["outcome"].astype(int).to_numpy()
test_features_mapped = default_mapper.transform(test_features)

Now let's plot the Type1-Type2 graph and select threholds based on our requirements:

In [ ]:
test_features.columns

In [ ]:
precision, recall, thresh = precision_recall_curve(test_outcomes, default_lr.predict_proba(test_features_mapped)[:,1])

))))))))))))))))

## Option 1: Determine the thresholds by hand, looking at the Type2-Type1 errors plot

In [ ]:
precision, recall, thresh = make_precision_recall_curve(
    test,
    default_pp, # to be built from the mapper as step 0 and the lr as step 1
    default_conf
)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

type_2 = 1 - recall
type_1 = 1 - precision

ax.scatter(
    type_1,
    type_2,
    label = "logreg"
);

plt.axhline(y=0.37, color='orange', linestyle='-')
plt.axvline(x=0.07, color='red', linestyle='-')

ax.set_xlabel('Type 1 error', fontsize=16)
ax.set_ylabel('Type 2 error', fontsize=16)

ax.legend(fontsize=18);

On peut sélectionner un seuil F1 (rouge) et un seuil F2 (orange) à partir de cette courbe:
- pour la liste F1, on cherche à maximiser la précision, puisqe la liste en question doit être la plus pertinent possible et contenir des entreprises "prioritaires". Cette liste minimise donc les faux positifs, favorise la précision et minimise donc l'erreur de type 1.
- la liste F2 doit se charger de minimiser les faux négatifs, et est donc plus orienter sur la maximisation du recall. Il s'agit ici de minimiser l'erreur de type 2.

Pour $t_{F1}$, on choisit une valeur qui donne une erreur de type 1 petite tout en étant la plus indulgente possible pour l'erreur de type 2. C'est typiquement ce qu'un point de courbure maximale va réaliser. Ici $t_{F1}=0.07$ semble assez satisfaisant (ligne verticale rouge ci-dessus).

Pour $t_{F2}$, on choisir au contraire une valeur de courbe (négative) minimale, tout en satisfaisant déjà une erreur de type 2 suffisament faible. Ici $t_{F2}=0.37$ semble assez satisfaisant (ligne horizontale orange ci-dessus).

## Option 2: Select thresholds by maximising a f-beta (with beta specified for each alert level)

In [ ]:
# A tester
(t_F1, t_F2) = make_thresholds_from_fbeta(
    test,
    default_pp,
    default_conf,
    beta_F1 = 0.5,
    beta_F2 = 2
)

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $\beta=0.5$ - Optimal threshold: $t_{F1}=0.876$ - $f_{0.5}=0.683$<br>
F2 - $\beta=2$ - Optimal threshold: $t_{F2}=0.220$ - $f_{2}=0.495$

In [ ]:
evaluate(
    default_pp,
    test,
    beta=2,
    thresh=0.22
)

((((((((((((((((<br>
Local code

In [ ]:
f_beta_rec = []
f_beta_prc = []

for thr_id, thr in enumerate(thresh.tolist()):
    f_beta_rec.append(
        fbeta_score(y_true=test_outcomes, y_pred=(default_lr.predict_proba(test_features_mapped)[:,1]>=thr), beta=2)
    )
    f_beta_prc.append(
        fbeta_score(y_true=test_outcomes, y_pred=(default_lr.predict_proba(test_features_mapped)[:,1]>=thr), beta=0.6)
    )
    
f_beta_rec = np.array(f_beta_rec)
f_beta_prc = np.array(f_beta_prc)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.plot(
    thresh,
    f_beta_rec,
    color="r",
    label="beta=2"
)

ax.plot(
    thresh,
    f_beta_prc,
    color="b",
    label="beta=0.6"
)

ax.set_xlabel("Decision threshold", fontsize=18)
ax.set_ylabel("f_beta", fontsize=18)

ax.legend(fontsize=18);

In [ ]:
t_F1_id = np.argmax(f_beta_prc)
t_F1 = thresh[t_F1_id]

t_F2_id = np.argmax(f_beta_rec)
t_F2 = thresh[t_F2_id]

))))))))))))))))

In [ ]:
t_F1

In [ ]:
t_F2

## 3. Select threshold by specifying a minimal precision for T1 alerts and a minimal recall for T1 + T2 alerts

In [ ]:
# A tester
(t_F1_c, t_F2_c) = make_thresholds_from_conditions(
    test,
    default_pp,
    default_conf,
    min_precision_F1 = 0.93,
    min_recall_F2 = 0.63
)

In [ ]:
min_precision_F1 = 0.93
min_recall_F2 = 0.63

# Find the minimum threshold such that precision is greater than min_precision_F1
t_F1_id = np.argmax(precision>=min_precision_F1)
t_F1_c = thresh[t_F1_id]

# Find the maximum threshold such that recall is greater than min_recall_F2
t_F2_id = np.argmax(recall[::-1][:-1]>=min_recall_F2)
t_F2_c = thresh[::-1][t_F2_id]

In [ ]:
t_F1_c

In [ ]:
t_F2_c

Sur le dataset de test chargé depuis /home/simon.lebastard/predictsignauxfaibles/data/prod_mars2021/2103_prod_data_test.csv (250k):<br>
F1 - $t_{F1}=0.921$ garantit une précision de 93% pour la liste F1 (rouge)<br>
F2 - $t_{F2}=0.124$ garantit un recall de 63% pour la liste F2 (orange)

## Research - Can we build a threshold selection criterion based on the variations of recall % threhold?

In [ ]:
# Let's compute the first-order derivative of the type-2 error
fod_recall = (recall[:-1] - np.roll(recall[:-1],1))/(thresh - np.roll(thresh,1))
fod_recall = fod_recall[1:]

In [ ]:
plt.hist(fod_recall)

In [ ]:
plt.plot(thresh)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(
    thresh[:-81],
    fod_recall[:-80],
    label = "logreg"
);

In [ ]:
recalL_maxvar_id = np.argmin(fod_recall[:-80])
recall_maxvar = fod_recall[recalL_maxvar_id]

recall_maxvar_precision = precision[recalL_maxvar_id]
recall_maxvar_recall = recall[recalL_maxvar_id]
recall_maxvar_thresh = thresh[recalL_maxvar_id]

In [ ]:
recall_maxvar_thresh

In [ ]:
recall_maxvar_recall

In [ ]:
recall_maxvar_precision

For now, we'll stick to manual selection based on the two types of error above, or maybe based on the maximisation of Fbeta

## Building json to be sent to collection Scores on MongoDB

In [ ]:
scores_full = scores_raw[["siret","siren","predicted_probability_full"]]
scores_small = scores_raw[["siret","siren","predicted_probability_small"]]

In [ ]:
scores_full.rename(columns={"predicted_probability_full": "predicted_probability"}, inplace=True)
scores_small.rename(columns={"predicted_probability_small": "predicted_probability"}, inplace=True)

In [ ]:
scores_full

In [ ]:
scores_small

In [ ]:
scores = merge_models(model_list = [scores_full, scores_small])

In [ ]:
scores